# Вычисление электроёмкости конденсатора 

In [2]:
%use dataframe
import kotlin.math.*

In [3]:
fun Double.round(decimals: Int): Double {
    var multiplier = 1.0
    repeat(decimals) { multiplier *= 10 }
    return round(this * multiplier) / multiplier
}

In [4]:
val df = DataFrame.read("data.csv")
df

n0,n1,n2,n3,n4
43,32,13,11,46
40,30,13,10,45
41,30,13,9,45
42,31,14,10,44
42,30,14,10,45


Вычисление среднего значения

In [5]:
val average = df.mean()
average

n0,n1,n2,n3,n4
41.600000,30.600000,13.400000,10.000000,45.000000


In [6]:
average.toMap()

{n0=41.6, n1=30.6, n2=13.4, n3=10.0, n4=45.0}

Значения для дальнейших вычислений

In [7]:
val c0 = 4700  // пикофарад
val n0 = average["n0"] as Double // сдвиг
val U = 17 // вольт

Вычислим постоянную гальванометра

In [8]:
val K = ((c0 * U) / n0).round(2)
println("${K} пКл/дел")

1920.67 пКл/дел


In [9]:
val ns = (average.toDataFrame().remove("n0").toMap().values.flatten() as List<Double>)
ns

[30.6, 13.4, 10.0, 45.0]

Вычисляем емкости конденсаторов

In [11]:
val cs = ns.map{((c0 * it)/n0).round(2)}

cs.forEachIndexed { index, value ->
    println("C${index + 1} = ${value}")
}

C1 = 3457.21
C2 = 1513.94
C3 = 1129.81
C4 = 5084.13


#### Вычисляемые значения

Параллельное соединение

In [10]:
cs[0] + cs[1]

4971.15

Последоватеьное соединение

In [11]:
((cs[0] * cs[1])/(cs[0] + cs[1])).round(2)

1052.88

## Вычисление погрешностей

### Систематическая погрешность

In [17]:
val thetaN = 0.75
val thetas = cs.mapIndexed{ index, C ->  C /* * Math.pow(10.0, -12.0) */ * (thetaN/n0 + thetaN/ns[index]) }
thetas

[147.06504642722473, 112.03003910734787, 105.10492067307692, 176.39649759615386]

#### Вычисляемые значения

Последовательное соединение

In [24]:
Math.pow(thetas[1], 2.0) / Math.pow(thetas[0] + thetas[1],2.0) * thetas[0] + Math.pow(thetas[0], 2.0) / Math.pow(thetas[0] + thetas[1],2.0) * thetas[1] 

63.58940721925605

Паралельное соединение

In [25]:
thetas[0] + thetas[1]

259.0950855345726

In [26]:
147.07 + 112.03

259.1

### Случайная погрешность

In [69]:
val Sn0 = Math.sqrt(df.n0.map{ Math.pow(it - n0, 2.0) }.sum() / 4.0)
Sn0

1.140175425099138

In [70]:
val Sn1 = Math.sqrt(df.n1.map{ Math.pow(it - ns[0], 2.0) }.sum() / 4.0)
Sn1

0.8944271909999159

In [71]:
val Sn2 = Math.sqrt(df.n2.map{ Math.pow(it - ns[1], 2.0) }.sum() / 4.0)
Sn2

0.5477225575051662

In [72]:
val Sn3 = Math.sqrt(df.n3.map{ Math.pow(it - ns[2], 2.0) }.sum() / 4.0)
Sn3

0.7071067811865476

In [73]:
val Sn4 = Math.sqrt(df.n4.map{ Math.pow(it - ns[3], 2.0) }.sum() / 4.0)
Sn4

0.7071067811865476

In [74]:
Sn0 / Math.sqrt(5.0)

0.5099019513592785